In [1]:
import pandas as pd
import plotly.express as px

In [2]:
data = pd.read_csv("amazon.csv")
df = pd.read_csv("amazon.csv")

In [3]:
max_depth = 7 # Define the maximum depth of categories you expect

# Split categories with max split
category_layers = df['category'].str.split('|', expand=True, n=max_depth - 1)

# Fill shorter entries with None (or "")
category_layers = category_layers.apply(lambda row: row.tolist() + [None]*(max_depth-len(row)), axis=1, result_type='expand')

# Name the columns
column_names = [f'category_{i+1}' for i in range(max_depth)]
category_layers.columns = column_names

# Combine with the main DataFrame
df = pd.concat([df, category_layers], axis=1)

for i in ["discounted_price","actual_price","rating_count","discount_percentage"]:
  df[i] = df[i].str.replace('₹', '', regex=False)
  df[i] = df[i].str.replace(',', '', regex=False)
  df[i] = df[i].str.replace('%', '', regex=False)
  df[i] = df[i].astype(float)

In [4]:
df = df.drop(columns = ['img_link','product_link'])

In [5]:
df

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,...,review_id,review_title,review_content,category_1,category_2,category_3,category_4,category_5,category_6,category_7
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,64.0,4.2,24269.0,High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...",...,"R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,None,None
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,199.0,349.0,43.0,4.0,43994.0,"Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...",...,"RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,None,None
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,199.0,1899.0,90.0,3.9,7928.0,【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...",...,"R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,None,None
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,329.0,699.0,53.0,4.2,94363.0,The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...",...,"R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,None,None
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,154.0,399.0,61.0,4.2,16905.0,[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...",...,"R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...,379.0,919.0,59.0,4,1090.0,SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,"AHITFY6AHALOFOHOZEOC6XBP4FEA,AFRABBODZJZQB6Z4U...",...,"R3G3XFHPBFF0E8,R3C0BZCD32EIGW,R2EBVBCN9QPD9R,R...","Received the product without spanner,Excellent...","I received product without spanner,Excellent p...",Home&Kitchen,Kitchen&HomeAppliances,WaterPurifiers&Accessories,WaterPurifierAccessories,None,None,None
1461,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,2280.0,3045.0,25.0,4.1,4118.0,"230 Volts, 400 watts, 1 Year","AFG5FM3NEMOL6BNFRV2NK5FNJCHQ,AGEINTRN6Z563RMLH...",...,"R3DDL2UPKQ2CK9,R2SYYU1OATVIU5,R1VM993161IYRW,R...","ok,everything was good couldn't return bcoz I ...","ok,got everything as mentioned but the measuri...",Home&Kitchen,Kitchen&HomeAppliances,SmallKitchenAppliances,Rice&PastaCookers,None,None,None
1462,B009P2LIL4,Bajaj Majesty RX10 2000 Watts Heat Convector R...,"Home&Kitchen|Heating,Cooling&AirQuality|RoomHe...",2219.0,3080.0,28.0,3.6,468.0,International design and styling|Two heat sett...,"AG

In [6]:
df['category_1'].unique()

array(['Computers&Accessories', 'Electronics', 'MusicalInstruments',
       'OfficeProducts', 'Home&Kitchen', 'HomeImprovement', 'Toys&Games',
       'Car&Motorbike', 'Health&PersonalCare'], dtype=object)

In [7]:
for i in range(1,8):
  sales_per_category = df.groupby(f'category_{i}').size().reset_index(name='product_sales')

  fig = px.bar(sales_per_category, x=f'category_{i}', y='product_sales',
             title='SALES IN EACH CATEGORY',
             labels={f'category_{i}': 'Category', 'product_sales': 'Product Sales'},
             color=f'category_{i}',  # Color bars by year for better visualization
             text='product_sales')  # Display the count on top of each bar
  fig.show()

In [8]:
df['no_of_users'] = df['user_name'].str.count(',') + 1
df['total_sales_amt'] = df['no_of_users'] * df['discounted_price']
df['no_of_users'].unique()

array([8, 9, 4, 3, 1, 7, 5, 6, 2])

In [9]:
df

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,...,review_content,category_1,category_2,category_3,category_4,category_5,category_6,category_7,no_of_users,total_sales_amt
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,64.0,4.2,24269.0,High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...",...,Looks durable Charging is fine tooNo complains...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,None,None,8,3192.0
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,199.0,349.0,43.0,4.0,43994.0,"Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...",...,I ordered this cable to connect my phone to An...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,None,None,8,1592.0
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,199.0,1899.0,90.0,3.9,7928.0,【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...",...,"Not quite durable and sturdy,https://m.media-a...",Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,None,None,8,1592.0
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,329.0,699.0,53.0,4.2,94363.0,The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...",...,"Good product,long wire,Charges good,Nice,I bou...",Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,None,None,8,2632.0
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,154.0,399.0,61.0,4.2,16905.0,[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...",...,"Bought this instead of original apple, does th...",Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,None,None,8,1232.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...,379.0,919.0,59.0,4,1090.0,SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,"AHITFY6AHALOFOHOZEOC6XBP4FEA,AFRABBODZJZQB6Z4U...",...,"I received product without spanner,Excellent p...",Home&Kitchen,Kitchen&HomeAppliances,WaterPurifiers&Accessories,WaterPurifierAccessories,None,None,None,8,3032.0
1461,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,2280.0,3045.0,25.0,4.1,4118.0,"230 Volts, 400 watts, 1 Year","AFG5FM3NEMOL6BNFRV2NK5FNJCHQ,AGEINTRN6Z563RMLH...",...,"ok,got everything as mentioned but the measuri...",Home&Kitchen,Kitchen&HomeAppliances,SmallKitchenAppliances,Rice&PastaCookers,None,None,None,8,18240.0
1462,B009P2LIL4,Bajaj Majesty RX10 2000 Watts Heat Convector R...,"Home&Kitchen|Heating,Cooling&AirQuality|RoomHe...",2219.0,3080.0,28.0,3.6,468.0,International design and styling|Two heat sett...,"AGVPWCMAHYQWJOQKMUJN4DW3KM5Q,AF4Q3E66MY4SR7YQZ...",...,"plastic but cool body ,u have to find sturdy s...",Home&Kitchen,"Heating,Cooling&AirQuality",RoomHeaters,HeatConvectors,None,None,None,8,17752.0
1463,B00J5DYCCA,Havells Ventil Air DSP 230mm Exhaust Fan (Pist...,"Home&Kitchen|Heating,Cooling&AirQuality|Fans|E...",1399.0,1890.0,26.0,4,8031.0,Fan sweep area: 230 MM ; Noise level: (40 - 45...,"AF2JQCLSCY3QJATWUNNHUSVUPNQQ,AFDMLUXC5LS5RXDJS...",...,I have installed this in my kitchen working fi...,Home&Kitchen,"Heating,Cooling&AirQuality",Fans,ExhaustFans,None,None,None,8,11192.0


In [10]:
for i in range(1,8):
  sales_per_category = df.groupby(f'category_{i}')['total_sales_amt'].sum().reset_index(name='product_sales')
  # Create a bar plot using Plotly
  fig = px.bar(sales_per_category, x=f'category_{i}', y='product_sales',
             title='SALES IN EACH CATEGORY',
             labels={f'category_{i}': 'Category', 'product_sales': 'Product Sales(in ₹)'},
             color=f'category_{i}',  # Color bars by year for better visualization
             text='product_sales')  # Display the count on top of each bar
  fig.show()

In [11]:
df.columns

Index(['product_id', 'product_name', 'category', 'discounted_price',
       'actual_price', 'discount_percentage', 'rating', 'rating_count',
       'about_product', 'user_id', 'user_name', 'review_id', 'review_title',
       'review_content', 'category_1', 'category_2', 'category_3',
       'category_4', 'category_5', 'category_6', 'category_7', 'no_of_users',
       'total_sales_amt'],
      dtype='object')

In [12]:
fig2 = px.box(df, y="discount_percentage", title="Distribution of Discount Percentages")
fig2.show()

In [13]:
fig3 = px.scatter(df, x="actual_price", y="discount_percentage",
                  title="Discount Percentage vs. Actual Price",
                  size="discount_percentage", color="discount_percentage",
                  color_continuous_scale="viridis")
fig3.show()

In [14]:
df[['category_1', 'category_2', 'category_3','category_4','category_5','category_6','category_7']] = df[['category_1', 'category_2', 'category_3','category_4','category_5','category_6','category_7']].fillna("None")

fig5 = px.treemap(df, path=['category_1', 'category_2', 'category_3','category_4','category_5','category_6','category_7'],
                   values="total_sales_amt", title="Sales Distribution Across Categories",
                   color="total_sales_amt", color_continuous_scale="plasma")
fig5.show()

In [15]:
fig = px.scatter(df, x="discount_percentage", y="rating",
                 size="total_sales_amt", color="category",
                 title="Impact of Discounts on Ratings & Sales",
                 hover_data=['product_name'])
fig.show()


In [16]:
df.isnull().sum()

,0
product_id,0
product_name,0
category,0
discounted_price,0
actual_price,0
discount_percentage,0
rating,0
rating_count,2
about_product,0
user_id,0


In [17]:
df = df.dropna(subset = ['rating'])
fig = px.parallel_coordinates(df, dimensions=[ 'discount_percentage','rating_count', 'total_sales_amt'],
                              color="total_sales_amt", color_continuous_scale=px.colors.sequential.Plasma)
fig.show()


In [18]:
df.columns

Index(['product_id', 'product_name', 'category', 'discounted_price',
       'actual_price', 'discount_percentage', 'rating', 'rating_count',
       'about_product', 'user_id', 'user_name', 'review_id', 'review_title',
       'review_content', 'category_1', 'category_2', 'category_3',
       'category_4', 'category_5', 'category_6', 'category_7', 'no_of_users',
       'total_sales_amt'],
      dtype='object')

In [19]:
df.dtypes

,0
product_id,object
product_name,object
category,object
discounted_price,float64
actual_price,float64
discount_percentage,float64
rating,object
rating_count,float64
about_product,object
user_id,object


In [20]:
# Import necessary libraries
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Fill missing values for text columns with an empty string
df['about_product'] = df['about_product'].fillna('')
df['review_content'] = df['review_content'].fillna('')
df['review_title'] = df['review_title'].fillna('')
for col in ['category_1', 'category_2', 'category_3','category_4','category_5','category_6','category_7']:
  df[col] = df[col].replace('None','Unknown')
df['category'] = df['category'].replace('None','Unknown')

# Convert numerical columns to float (if stored as string)
numeric_cols = ['discounted_price', 'actual_price', 'discount_percentage', 'rating', 'rating_count', 'total_sales_amt', 'no_of_users']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric, set invalid values as NaN

# Now, fill missing values with the median
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

# Normalize numerical columns
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

print(" Data types fixed and numeric values converted successfully!")



 Data types fixed and numeric values converted successfully!


In [21]:
print(df.isnull().sum())  # Check for any remaining NaNs
category_cols = ['category', 'category_1', 'category_2', 'category_3', 'category_4', 'category_5', 'category_6', 'category_7']
df[category_cols] = df[category_cols].astype(str)
df['text_data'] = df['about_product'] + " " + df['review_content'] + " " + df['review_title']
df['text_data'] = df['text_data'].apply(lambda x: x if x.strip() != '' else 'No description available')


product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           0
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
category_1             0
category_2             0
category_3             0
category_4             0
category_5             0
category_6             0
category_7             0
no_of_users            0
total_sales_amt        0
dtype: int64


In [22]:
#CONTENT - BASED FILTERING
from sklearn.feature_extraction.text import TfidfVectorizer  # For text vectorization
from sklearn.metrics.pairwise import cosine_similarity

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text_data'])

# Compute Cosine Similarity between products
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to get similar products based on text similarity
def get_similar_products(product_id, top_n=5):
    try:
        idx = df[df['product_id'] == product_id].index[0]  # Get product index
        sim_scores = list(enumerate(cosine_sim[idx]))  # Get similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]  # Sort and get top N
        product_indices = [i[0] for i in sim_scores]
        return data.iloc[product_indices][['product_id', 'product_name', 'category', 'discounted_price','actual_price','rating']]
    except IndexError:
        return " Product ID not found in dataset"


In [23]:
# Example Usage
get_similar_products(product_id='B009P2LIL4', top_n=7)

,product_id,product_name,category,discounted_price,actual_price,rating
1097,B009P2LITG,Bajaj Majesty RX11 2000 Watts Heat Convector R...,"Home&Kitchen|Heating,Cooling&AirQuality|RoomHe...","₹2,169","₹3,279",4.1
1347,B09M3F4HGB,Borosil Volcano 13 Fin Oil Filled Radiator Roo...,"Home&Kitchen|Heating,Cooling&AirQuality|RoomHe...","₹9,495","₹18,990",4.2
1458,B07J9KXQCC,"Libra Room Heater for Home, Room Heaters Home ...","Home&Kitchen|Heating,Cooling&AirQuality|RoomHe...",₹949,"₹2,299",3.6
1123,B01N1XVVLC,"Bajaj OFR Room Heater, 13 Fin 2900 Watts Oil F...","Home&Kitchen|Heating,Cooling&AirQuality|RoomHe...","₹9,590","₹15,999",4.1
1282,B09H34V36W,Inalsa Electric Fan Heater Hotty - 2000 Watts ...,"Home&Kitchen|Heating,Cooling&AirQuality|RoomHe...","₹1,349","₹2,495",3.8
1340,B09MQ9PDHR,SaiEllin Room Heater For Home 2000 Watts Room ...,"Home&Kitchen|Heating,Cooling&AirQuality|RoomHe...",₹979,"₹1,999",3.9
1105,B009P2LK80,Bajaj Deluxe 2000 Watts Halogen Room Heater (S...,"Home&Kitchen|Heating,Cooling&AirQuality|RoomHe...","₹1,409","₹1,639",3.7


In [24]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
# If user_id column has comma-separated values, split them and explode into multiple rows
df['user_id'] = df['user_id'].str.split(',')
df = df.explode('user_id')

# 1. Build the User-Item Matrix using pivot_table to handle duplicate entries
ratings_matrix = df.pivot_table(index='user_id',
                                columns='product_id',
                                values='rating',
                                aggfunc='mean').fillna(0)

# 2. Normalize the ratings by subtracting the mean rating for each user
mean_user_rating = ratings_matrix.mean(axis=1)
ratings_demeaned = ratings_matrix.sub(mean_user_rating, axis=0)
# Convert the demeaned ratings DataFrame to a NumPy array for SVD
ratings_demeaned_values = ratings_demeaned.values



In [25]:
df[['product_id','user_id','rating']]

,product_id,user_id,rating
0,B07JW9H4J1,AG3D6O4STAQKAY2UVGEUV46KN35Q,0.733333
0,B07JW9H4J1,AHMY5CWJMMK5BJRBBSNLYT3ONILA,0.733333
0,B07JW9H4J1,AHCTC6ULH4XB6YHDY6PCH2R772LQ,0.733333
0,B07JW9H4J1,AGYHHIERNXKA6P5T7CZLXKVPT7IQ,0.733333
0,B07JW9H4J1,AG4OGOFWXJZTQ2HKYIOCOY3KXF2Q,0.733333
...,...,...,...
1464,B01486F4G6,AHXCDNSXAESERITAFELQABFVNLCA,0.766667
1464,B01486F4G6,AGRZD6CHLCUNOLMMIMIHUCG7PIFA,0.766667
1464,B01486F4G6,AFQZVGSOSOJHKFQQMCEI4725QEKQ,0.766667
1464,B01486F4G6,AEALVGXXIP46OZVXKRUXSDWZJMEA,0.766667


In [26]:
print("Unique rating values:", data['rating'].unique())


Unique rating values: ['4.2' '4.0' '3.9' '4.1' '4.3' '4.4' '4.5' '3.7' '3.3' '3.6' '3.4' '3.8'
 '3.5' '4.6' '3.2' '5.0' '4.7' '3.0' '2.8' '4' '3.1' '4.8' '2.3' '|' '2'
 '3' '2.6' '2.9']


In [27]:
stacked = ratings_matrix.stack()

# Filter out the zeros
non_zero_ratings = stacked[stacked != 0]

print(non_zero_ratings)

user_id                       product_id
AE22E2AXODSPNK3EBIHNGYS5LOSA  B0B31FR4Y2    0.566667
AE22MK2NXQD3ZARLIOL3SLD4GU6A  B0B8XNPQPN    0.733333
AE22Y3KIS7SE6LI3HE2VS6WWPU4Q  B07ZR4S1G4    0.800000
AE23RS3W7GZO7LHYKJU6KSKVM4MQ  B00HVXS7WC    0.733333
AE23WGYTUMB5R6JJMBU4V43JIW7Q  B09163Q5CD    0.700000
                                              ...   
AHZXNS63DN6MZDH3WSKYRLWSG3DA  B08Y5QJTVK    0.800000
AHZYN7O73FJWRPUQGZM5BEAZ3A4A  B08TTRVWKY    0.766667
AHZZFBL24XXVLW6H44MOB6LBHH5A  B0B9BD2YL4    0.833333
AHZZISWHBQV4TL3MKOJTH22IUNIQ  B095X38CJS    0.766667
AHZZXJWETMZR7SH7C22KVUT7VHAA  B08WLY8V9S    0.833333
Length: 10602, dtype: float64


In [28]:
# Adjust the number of latent factors (k) as needed; here, k=50 is used.
U, sigma, Vt = svds(ratings_demeaned_values, k=50)
sigma = np.diag(sigma)  # Convert sigma into a diagonal matrix

# 4. Reconstruct the predicted ratings matrix
predicted_ratings = np.dot(np.dot(U, sigma), Vt) + mean_user_rating.values.reshape(-1, 1)
predicted_df = pd.DataFrame(predicted_ratings, index=ratings_matrix.index, columns=ratings_matrix.columns)

print("✅ SVD Applied & Predictions Generated!")

✅ SVD Applied & Predictions Generated!


In [29]:
# 5. Function to Recommend Products for a Given User
def recommend_products(user_id, top_n=5):
    if user_id not in predicted_df.index:
        return "❌ User not found!"

    # Get predicted ratings for the user and sort them in descending order
    sorted_ratings = predicted_df.loc[user_id].sort_values(ascending=False)

    # Get top N product IDs
    top_product_ids = sorted_ratings.index[:top_n]

    # Retrieve product details from the original dataframe.
    # If a product appears multiple times in df, consider dropping duplicates.
    recommended_products = data[data['product_id'].isin(top_product_ids)].drop_duplicates('product_id')

    return recommended_products[['product_id', 'product_name', 'category', 'discounted_price','actual_price','rating']]

In [30]:
recommend_products('AG3D6O4STAQKAY2UVGEUV46KN35Q',top_n=5)

,product_id,product_name,category,discounted_price,actual_price,rating
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",4.2
42,B07JW1Y6XV,Wayona Nylon Braided 3A Lightning to USB A Syn...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",4.2
80,B07LGT55SJ,Wayona Usb Nylon Braided Data Sync And Chargin...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",4.2
106,B07JGDB5M1,Wayona Nylon Braided 2M / 6Ft Fast Charge Usb ...,Computers&Accessories|Accessories&Peripherals|...,₹449,"₹1,299",4.2
220,B07JH1CBGW,Wayona Nylon Braided Usb Syncing And Charging ...,Computers&Accessories|Accessories&Peripherals|...,₹649,"₹1,999",4.2


In [31]:
#HYBRID RECOMMENDATION SYSTEM
!pip install surprise
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

def content_based_recommendations(df, product_id, top_n):
    # TF-IDF Vectorization on 'about_product' column
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['about_product'].fillna(''))

    # Compute Cosine Similarity
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Get similar products
    idx = df[df['product_id'] == product_id].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]

    product_indices = [i[0] for i in sim_scores]
    return df.iloc[product_indices][['product_id', 'product_name', 'category']]

def collaborative_recommendations(df, user_id, top_n):
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(df[['user_id', 'product_id', 'rating']], reader)
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

    model = SVD(random_state = 42)
    model.fit(trainset)

    unique_products = df['product_id'].unique()
    predictions = [model.predict(user_id, pid).est for pid in unique_products]

    recommended_products = np.argsort(predictions)[-top_n:][::-1]
    return df.iloc[recommended_products][['product_id', 'product_name', 'category']]

def hybrid_recommendation(df, user_id, product_id, top_n, alpha=0.5 ):
    content_recs = content_based_recommendations(df, product_id, top_n)
    collab_recs = collaborative_recommendations(df, user_id, top_n)

    merged = pd.concat([content_recs, collab_recs]).drop_duplicates().reset_index(drop=True)
    merged['hybrid_score'] = alpha * np.arange(len(merged)) + (1-alpha) * np.random.rand(len(merged))

    return merged.sort_values(by='hybrid_score', ascending=False)

# Example usage:


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505172 sha256=e2a4ea68b29137908df14cedc5cc5fbf080c1415a6fc5111c31efb857cc93ec2
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [33]:
hybrid_recommendation(df, user_id="AE22Y3KIS7SE6LI3HE2VS6WWPU4Q", product_id="B07JW9H4J1",top_n = int(input()))

5


,product_id,product_name,category,hybrid_score
5,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,2.747270
4,B082T6GVLJ,Amazon Basics New Release Nylon USB-A to Light...,Computers&Accessories|Accessories&Peripherals|...,2.345449
3,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,1.802429
2,B08QX1CC14,SKYWALL 81.28 cm (32 inches) HD Ready Smart LE...,"Electronics|HomeTheater,TV&Video|Televisions|S...",1.494303
1,B09QGZFBPM,Wayona Type C To Type C Long Fast Charging Cab...,Computers&Accessories|Accessories&Peripherals|...,0.887042
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,0.038260


In [34]:
pip install transformers torch datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [35]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import pandas as pd


In [36]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [37]:
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [38]:
def get_sentiment(text):
    try:
        result = sentiment_pipeline(text[:512])  # BERT has a 512 token limit
        return result[0]['label']  # Extract sentiment label
    except:
        return "Error"

data['sentiment'] = data['review_content'].astype(str).apply(get_sentiment)
sentiment_mapping = {
    "1 star": "Very Negative",
    "2 stars": "Negative",
    "3 stars": "Neutral",
    "4 stars": "Positive",
    "5 stars": "Very Positive"
}
data['sentiment'] = data['sentiment'].map(sentiment_mapping)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [39]:
data[['review_content', 'sentiment']]

,review_content,sentiment
0,Looks durable Charging is fine tooNo complains...,Positive
1,I ordered this cable to connect my phone to An...,Positive
2,"Not quite durable and sturdy,https://m.media-a...",Neutral
3,"Good product,long wire,Charges good,Nice,I bou...",Positive
4,"Bought this instead of original apple, does th...",Neutral
...,...,...
1460,"I received product without spanner,Excellent p...",Positive
1461,"ok,got everything as mentioned but the measuri...",Negative
1462,"plastic but cool body ,u have to find sturdy s...",Positive
1463,I have installed this in my kitchen working fi...,Positive


In [61]:
from sklearn.metrics import accuracy_score
import math
data['rating'] = pd.to_numeric(data['rating'], errors='coerce')

# Drop any NaN values (optional, if needed)
data = data.dropna(subset=['rating'])
data['rounded_rating'] = data['rating'].apply(math.floor).astype(int)
mapping = {
    1: "Very Negative",
    2: "Negative",
    3: "Neutral",
    4: "Positive",
    5: "Very Positive"
}
data['expected_sentiment'] = data['rounded_rating'].map(mapping)
print("Unique values in predicted sentiment:", data['sentiment'].unique())
print("Unique values in expected sentiment:", data['expected_sentiment'].unique())



Unique values in predicted sentiment: ['Positive' 'Neutral' 'Very Positive' 'Negative' 'Very Negative']
Unique values in expected sentiment: ['Positive' 'Neutral' 'Very Positive' 'Negative']


In [62]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(data['expected_sentiment'], data['sentiment'])
print(f"Sentiment Prediction vs Ratings Accuracy: {accuracy:.2%}")

Sentiment Prediction vs Ratings Accuracy: 45.08%


In [47]:
data['rating_count'] = pd.to_numeric(data['rating'], errors='coerce')
df_filtered = data[data['rating_count'].astype(float) > 6]  # Consider only products with more than 6 reviews

# Create scatter plot
fig = px.scatter(
    df_filtered,
    x="rating_count",
    y="rating",
    color="rating",
    size="rating_count",
    hover_data=["product_name", "rating_count", "rating"],
    title="Interactive Scatter Plot of Products Based on Ratings",
    labels={"rating_count": "Number of Reviews", "rating": "Average Rating"},
    color_continuous_scale="Viridis"
)

# Show interactive plot
fig.show()